In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mirae = pd.read_csv('/content/drive/MyDrive/미래에셋/mirae_stockname.csv')

In [ ]:
#매수 칼럼들만 따질 것이므로 매수 관련 종목 칼럼만 따로 빼온다.
mirae_maesu_list = []
mirae_columns = list(mirae.columns)
for x in mirae_columns:
  if '주식매수' in x:
    if len(x) > 13:
      mirae_maesu_list.append(x)
mirae = mirae[mirae_maesu_list]

In [ ]:
#업종 정보 불러오기
upjong = pd.read_csv('/content/drive/MyDrive/미래에셋/전처리/data/upjong.csv')
#매수 종목 이름에 따른 업종 분류(소분류, 중분류)만 뽑아오기 위해 new_df를 만든다.
new_df = pd.DataFrame([])

#RAM의 크기 때문에 각 달마다 매수 탑 3 종목, 금액, 업종(중분류, 소분류)만 따로 dataframe을 만듬
for maesu in mirae_maesu_list:
  if '금액' not in maesu:
    new_df = pd.concat([new_df, pd.merge(mirae, upjong, how = 'left', left_on = maesu, right_on = '회사명')[[maesu, '중분류','소분류']].rename(columns = {maesu:maesu, '중분류':maesu+'_중분류','소분류':maesu+'_소분류'})], axis = 1)
  else:
    new_df = pd.concat([new_df, mirae[[maesu]]], axis = 1)

new_df.head()

In [ ]:
#더미 dict와 dataframe 만들어 놓기
zero_dict = {}
for i in  list(set(middle_df['중분류'])):
  zero_dict[i] = 0
zero_dict
chong_dict = {'총매수1위업종':[], '총매수1위금액':[],'총매수2위업종':[],'총매수2위금액':[],'총매수3위업종':[],'총매수3위금액':[]}
years = ['202105','202104','202103','202102','202101','202012','202011','202010','202009','202008','202007','202006']

In [ ]:
#종목명이 없거나 금액이 나와있지 않은 칼럼은 모두 0으로 채우기
new_df = new_df.fillna(value = 0)

In [ ]:
#총매수금액과 업종(중분류) 탑 3를 구하는 함수
def chong(numb):
  test_df = new_df.iloc[numb,:]
  cool_dict = zero_dict.copy()
  #업종(중분류)별 1년간 총 매수한 금액 구하기
  for i in years:
    for j in range(1,4):
      if test_df['주식매수'+str(j)+'위종목_'+i+'_중분류'] != 0:
        cool_dict[test_df['주식매수'+str(j)+'위종목_'+i+'_중분류']] += test_df['주식매수'+str(j)+'위금액_'+i]
      else:
        continue
  test_list = []
  #총 매수 금액 탑 3 금액과 업종(중분류) 구하기
  for i in sorted(cool_dict.values(), reverse = True)[0:3]:
    for j in cool_dict.keys():
      if i != 0 and cool_dict[j] == i:
        if [j, i] not in test_list:
          test_list.append([j, i])
          continue
      elif i == 0:
        test_list.append(['NaN',0])
        continue
  # print(test_list)
  # 새로운 딕셔러니에 넣어서 새롭게 dataframe만들 준비하기
  for i,j in enumerate(test_list[0:3]):
    chong_dict['총매수{}위업종'.format(str(i+1))].append(j[0])
    chong_dict['총매수{}위금액'.format(str(i+1))].append(j[1])
  # print(chong_dict)

In [ ]:
#전체 데이터에 적용하기
for i in range(0,new_df.shape[0]):
  chong(i)

In [ ]:
#새롭게 만들어진 데이터프레임 만들기
upjong_price = pd.DataFrame(data = chong_dict)
upjong_price.head()

In [ ]:
upjong_price.shape

In [ ]:
#총매수 탑 3 금액과 업종(중분류) 데이터프레임 저장하기
upjong_price.to_csv('/content/drive/MyDrive/upjong_price.csv')